<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/DocumentSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graphics-drive

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 49kB/s 
     |████████████████████████████████| 204kB 36.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=717fec4d6671bad3e800b238213ef2ecbe91b4d5edf49a22fcb405660e57be4a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
pip install mmh3

     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
import re
import numpy as np
import collections
import random
from operator import itemgetter

In [ ]:
import nltk
from nltk import ngrams
import mmh3

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import time
import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
def getUrls(baseUrl):
  driver.get(baseUrl)
  time.sleep(3)
  previous_height = driver.execute_script("return document.body.scrollHeight;") 
  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  
      time.sleep(3)
      new_height = driver.execute_script("return document.body.scrollHeight;")
      if new_height == previous_height:
          break 
      previous_height = new_height
      time.sleep(3)
      driver.execute_script("document.querySelector('.view-more-detail a').click();")
      time.sleep(3)
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  
      time.sleep(3)
      new_height = driver.execute_script("return document.body.scrollHeight;")
      if new_height == previous_height:
          break 
      previous_height = new_height
            
  elements = driver.find_elements_by_css_selector("h3[class$='-title'] a")

  elementUrls = [el.get_attribute("href") for el in elements]
  return elementUrls



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
links = []

baseUrls = ["https://kenh14.vn/star.chn", 
            "https://kenh14.vn/tv-show.chn", 
            "https://kenh14.vn/cine.chn", 
            "https://kenh14.vn/musik.chn",
            "https://kenh14.vn/beauty-fashion.chn",
            "https://kenh14.vn/xem-mua-luon.chn",
            "https://kenh14.vn/doi-song.chn",
            "https://kenh14.vn/xa-hoi.chn",
            "https://kenh14.vn/the-gioi.chn",
            "https://kenh14.vn/sport.chn",
            "https://kenh14.vn/hoc-duong.chn"]

for baseUrl in baseUrls:
  link = getUrls(baseUrl)
  links.extend(link)

In [ ]:
print(len(links))
for link in links:
  if link == 'javascript:void(0);':
    links.remove(link)

2338


In [ ]:
data = []
driver2 = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
for link in links:
  driver2.get(link)
  title = driver2.find_elements_by_css_selector(".kbwc-title")
  if len(title) == 0:
    title = ''
  else:
    title = driver2.find_elements_by_css_selector(".kbwc-title")[0].text
  
  sapo = driver2.find_elements_by_css_selector(".knc-sapo")
  if len(sapo) == 0:
    abstract = ''
  else:
    abstract = driver2.find_elements_by_css_selector(".knc-sapo")[0].text
  contents = driver2.find_elements_by_css_selector(".knc-content p")
  contents = [content.text for content in contents]
  content = ('').join(contents)   
  data.append({
      "title": title,
      "abstract": abstract,
      "content": content,
  })

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


KeyboardInterrupt: ignored

In [ ]:
for document in data:
  if(document['content'] == ''):
    data.remove(document)
print(len(data))
print(data[0]['content'])

1183
Sau nghi vấn "đường ai nấy đi" với Thiều Bảo Trâm và drama “trà xanh” của Hải Tú, mọi nhất cử nhất động của Sơn Tùng đều được cư dân mạng đặc biệt quan tâm. Tuy nhiên thời gian qua, nam ca sĩ gần như lặn mất tăm trên các trang mạng xã hội, cũng không hề có bất cứ động thái nào liên quan đến 2 người con gái trên. Anh cũng chỉ tham gia một vài show diễn hay sự kiện của nhãn hàng.Nhưng bất ngờ mới đây, sau một thời gian dài nghe ngóng và theo chân, chúng tôi đã ghi lại được hình ảnh của nam ca sĩ khi xuất hiện ở một phim trường. Và theo các nguồn tin đáng tin cậy, đây là quay MV cho sản phẩm âm nhạc tiếp theo chứ không phải là quay clip quảng cáo. Các ekip làm việc với Tùng lần này đều là ekip ruột đã gắn bó với anh qua các MV trước. Liệu với kết thúc mở của MV Chúng Ta Của Hiện Tại thì liệu đây có phải là phần tiếp theo, như là Chúng Ta Của Tương Lai chẳng hạn?Cụ thể, chúng tôi bắt gặp Sơn Tùng có mặt tại một phim trường tại Quận 2 - TP.HCM vào sáng sớm ngày 29/3. Sơn Tùng đến bằng 

In [ ]:
with open('/content/drive/MyDrive/sample-datasets/kenh14news.json', 'w') as f:
  jsonString = json.dumps(data)
  f.write(jsonString)
  f.close()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

readJSONDF = spark.read.json('/content/drive/MyDrive/sample-datasets/kenh14news.json')
readJSONDF.show(5)

+--------------------+--------------------+--------------------+
|            abstract|             content|               title|
+--------------------+--------------------+--------------------+
|Đây là lần hiếm h...|Sau nghi vấn "đườ...|Độc Quyền: Sơn Tù...|
|Khoảnh khắc tình ...|Những câu chuyện ...|Tim chính thức cô...|
|Mới ngày đầu thán...|Là cặp vợ chồng v...|Trường Giang gửi ...|
|“Điên nữ” Seo Ye ...|Vào chiều ngày 24...|Lâu lắm mới tái x...|
|Những diễn biến x...|Tối ngày 31/3, Ng...|Chưa hết drama: N...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
data = readJSONDF.select("content").collect()
print(data[0]['content'])

Sau nghi vấn "đường ai nấy đi" với Thiều Bảo Trâm và drama “trà xanh” của Hải Tú, mọi nhất cử nhất động của Sơn Tùng đều được cư dân mạng đặc biệt quan tâm. Tuy nhiên thời gian qua, nam ca sĩ gần như lặn mất tăm trên các trang mạng xã hội, cũng không hề có bất cứ động thái nào liên quan đến 2 người con gái trên. Anh cũng chỉ tham gia một vài show diễn hay sự kiện của nhãn hàng.Nhưng bất ngờ mới đây, sau một thời gian dài nghe ngóng và theo chân, chúng tôi đã ghi lại được hình ảnh của nam ca sĩ khi xuất hiện ở một phim trường. Và theo các nguồn tin đáng tin cậy, đây là quay MV cho sản phẩm âm nhạc tiếp theo chứ không phải là quay clip quảng cáo. Các ekip làm việc với Tùng lần này đều là ekip ruột đã gắn bó với anh qua các MV trước. Liệu với kết thúc mở của MV Chúng Ta Của Hiện Tại thì liệu đây có phải là phần tiếp theo, như là Chúng Ta Của Tương Lai chẳng hạn?Cụ thể, chúng tôi bắt gặp Sơn Tùng có mặt tại một phim trường tại Quận 2 - TP.HCM vào sáng sớm ngày 29/3. Sơn Tùng đến bằng ô tô 

In [ ]:
def preprocess(document):
    document = re.sub(r'[^\w\s]','',document)
    tokens = document.lower()
    tokens = tokens.split()
    return tokens

In [ ]:
doc_as_shingles = []

def shingling(document):
  tokens = preprocess(document['content'])
  shingles = ngrams(tokens, 3)
  doc_as_shingles.append(list(shingles))
  return doc_as_shingles

for document in data:
  shingling(document)
print(doc_as_shingles[0])

[('sau', 'nghi', 'vấn'), ('nghi', 'vấn', 'đường'), ('vấn', 'đường', 'ai'), ('đường', 'ai', 'nấy'), ('ai', 'nấy', 'đi'), ('nấy', 'đi', 'với'), ('đi', 'với', 'thiều'), ('với', 'thiều', 'bảo'), ('thiều', 'bảo', 'trâm'), ('bảo', 'trâm', 'và'), ('trâm', 'và', 'drama'), ('và', 'drama', 'trà'), ('drama', 'trà', 'xanh'), ('trà', 'xanh', 'của'), ('xanh', 'của', 'hải'), ('của', 'hải', 'tú'), ('hải', 'tú', 'mọi'), ('tú', 'mọi', 'nhất'), ('mọi', 'nhất', 'cử'), ('nhất', 'cử', 'nhất'), ('cử', 'nhất', 'động'), ('nhất', 'động', 'của'), ('động', 'của', 'sơn'), ('của', 'sơn', 'tùng'), ('sơn', 'tùng', 'đều'), ('tùng', 'đều', 'được'), ('đều', 'được', 'cư'), ('được', 'cư', 'dân'), ('cư', 'dân', 'mạng'), ('dân', 'mạng', 'đặc'), ('mạng', 'đặc', 'biệt'), ('đặc', 'biệt', 'quan'), ('biệt', 'quan', 'tâm'), ('quan', 'tâm', 'tuy'), ('tâm', 'tuy', 'nhiên'), ('tuy', 'nhiên', 'thời'), ('nhiên', 'thời', 'gian'), ('thời', 'gian', 'qua'), ('gian', 'qua', 'nam'), ('qua', 'nam', 'ca'), ('nam', 'ca', 'sĩ'), ('ca', 'sĩ', 'g

In [ ]:
doc_hashed = []

def hash_doc(shingles):
  shingles_hashed = []
  for shingle in shingles:
    hash = mmh3.hash(' '.join(shingle))
    shingles_hashed.append(hash)
  return shingles_hashed  

for document in doc_as_shingles:
  doc_hashed.append(list(hash_doc(document)));
print(doc_hashed[0])

[-1746844108, 702369737, 972516078, -1264324051, 1173796847, -464829193, -2064918166, -277923450, 2040637177, 449141927, 1031847797, 1543888877, 1725551768, -1435549711, 579250164, 179396251, 1251615328, -1823814317, -1727963975, 1359400131, -619484912, -994988327, 493137236, 1264506849, -1661251425, 111344418, 390811016, -727250005, 1263801915, -667993614, 1726155041, 1194300217, 1315547187, 522235349, -168691567, -1783960478, 557749690, -1243848402, 1407278963, -50575916, -1575532904, -1836368867, -1650761433, 863997528, -1746598233, 814061102, -270543002, 48920382, -1290086104, 978494104, 1155020624, 1244718118, -1087634058, 1463058596, -1730320940, -2105386422, -1078167163, -716238641, 2105328596, 1214650102, 876016949, 1002646256, 1665376247, -507225906, 380764185, 1931273201, -377414495, 1742284643, -528182654, -796337304, 2938686, -2067201109, 152621598, -1771027460, 23075706, -1260425408, 1349777495, -1818450372, -1664322326, 1834517994, 925717976, 919142184, -569213744, -79570

In [ ]:
start = time.time()
signatures = []
max_shingle_id = 2 ** 32 - 1

next_prime = 4294967311

def pick_random_coeffs(k):
    rand_list = []

    while k > 0:
        rand_index = random.randint(0, max_shingle_id)

        while rand_index in rand_list:
            rand_index = random.randint(0, max_shingle_id)

        rand_list.append(rand_index)
        k = k - 1
    return rand_list

coeff_a = pick_random_coeffs(100)
coeff_b = pick_random_coeffs(100)

for document in doc_hashed:
  signature =[]
  for i in range(0, 100):
    min_hash_code = next_prime + 1
    for shingle in document: 
      hash_code = (coeff_a[i] * shingle + coeff_b[i]) % next_prime
      if hash_code < min_hash_code:
        min_hash_code = hash_code
    signature.append(min_hash_code)
  signatures.append(signature)
end = time.time()
print(signatures[0])
print(len(signatures),len(signatures[0]))
print("Minhash completed in : {:.4f} sec".format(end - start))

[4730865, 1756329, 185507, 582957, 105249, 21431026, 603892, 4342713, 13096226, 5353753, 5154830, 2612507, 1071698, 8448415, 412773, 536391, 3016316, 4109662, 262111, 9474109, 4856037, 3846817, 4391962, 6689537, 5355316, 956855, 6602197, 8005386, 8492040, 2350613, 282603, 28368442, 5075658, 623305, 11715666, 4134007, 1003709, 626286, 2699789, 745662, 2787851, 799917, 6624602, 886813, 587701, 4478602, 2262206, 9329670, 4396291, 986296, 8272114, 166615, 7082295, 7439882, 5305015, 8242476, 1810597, 1064164, 8305911, 416928, 8293627, 7091528, 3169585, 981173, 4475455, 476939, 12524831, 14022669, 1147800, 5398905, 2181002, 7721351, 2833109, 2781660, 6718564, 3702232, 1724442, 4154787, 7472357, 5767705, 4063604, 425597, 2241416, 2710979, 8959255, 4127641, 2564374, 9852972, 5450551, 1385225, 2471460, 791727, 921076, 6441261, 6990465, 1264829, 15801560, 6500755, 3201156, 95238]
1183 100
Minhash completed in : 37.8691 sec


In [ ]:
sample_document = signatures[100]
def apply_LSH_technique(signatures,t,bands,rows):
    if bands * rows != len(signatures):
        print('split to bands error.')

    array_buckets = initialize_array_bucket(bands)

    # hash_funcs = pick_random_coeffs(10);

    candidates = {}
    
    i = 0
    for b in range(bands):
        buckets = array_buckets[b] 
        band = np.array(signatures)[i:i+rows,:]

        for col in range(band.shape[1]):    
            key = int(sum(band[:,col]) % len(buckets))
            buckets[key].append(col)
        i = i+rows
        for item in buckets:   
            if len(item) > 1:
                pair = (item[0], item[1])
                if pair not in candidates:
                    A = np.array(signatures)[:,item[0]]
                    B = np.array(signatures)[:,item[1]]
                    similarity = cosine_distance(A,B)
                    if similarity >= t:
                        candidates[pair] = similarity

    sort = sorted(candidates.items(),key=itemgetter(1), reverse=True)

    return candidates,sort


start = time.time()
candidates, sort = apply_LSH_technique(signatures[0:1180], 0.3, 59, 20)
end = time.time()
print(candidates)
print("LSH completed in: {:.4f} sec".format(end - start))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in long_scalars
  


{}
LSH completed in: 45.4882 sec


In [ ]:
sample_document = signatures[100]
def create_bands(signature_matrix):
    # Creates bands given the signature matrix.
    # Separates the signature_matrix rows (n in total)
    # to b bands each of which consists of r rows (b*r=n)

    bands = []
    # get the number of bands and the number of rows
    b, r = get_b_r(len(signature_matrix))
    row_to_add = 0

    # for each one of b bands
    for band in range(b):
        rows = []
        for row in range(r):
            # get r rows from signature_matrix
            rows.append(signature_matrix[row_to_add])
            row_to_add = row_to_add + 1
        # add r rows to this band
        bands.append([band, rows])
    return bands


def create_hash_table(bands, k):
    # Creates buckets given the bands.
    # Hashes each column of bands to some large number of buckets.
    # The hash function gives the position in which the column
    # will be placed in the buckets array.
    # We use a separate bucket array for each band, so columns  vector
    # with the same in different bands will not hash to the same bucket.

    buckets = []
    for i in range(len(bands)):
        row = []
        for j in range(k):
            row.append([])
        buckets.append(row)

    # hash function parameters
    # random.seed(0)
    a = random.randint(1, 99)
    # random.seed(0)
    b = random.randint(1, 99)
    for band in range(len(bands)):
        for column in range(len(bands[band][1][0])):
            sum = 0
            for band_len in range(len(bands[band][1])):
                sum = sum + bands[band][1][band_len][column]
            position_in_hashed = ((a * sum + b) % 99) % k
            buckets[band][position_in_hashed].append(column)
        #     print("\tposition -->", position_in_hashed)
        # print("end band\n")
    return buckets


def candidate_column_pairs(buckets):
    # The buckets array is a sparse array, so candidate_column_pairs function
    # returns only the candidate pairs of documents (zip array of buckets)

    candidate_pairs_list = []
    # for each bucket array (for row of buckets)
    for row in range(len(buckets)):
        row_candidates_matches = []
        # for each bucket in bucket array
        for column in range(len(buckets[row])):
            # if bucket contains 2 or more documents
            if len(buckets[row][column]) > 1:
                # add bucket to new zip bucket array
                row_candidates_matches.append(buckets[row][column])
        # add zip bucket array to candidate_pairs_list
        candidate_pairs_list.append(row_candidates_matches)
    return candidate_pairs_list


def jaccard_similarity(list1, list2):
    # calculates the Jaccard similarity of 2 lists

    s1 = set(list1)
    s2 = set(list2)
    return float(str(round(len(s1.intersection(s2)) / len(s1.union(s2)), 3)))


def document_similarities(cand_pairs, sign_mtrx, docs):
    # calculates the Jaccard similarity of all pairs

    similarities = []
    # for each bucket array in candidate pairs list
    for bucket_array in range(len(cand_pairs)):
        # for each bucket in bucket array
        for bucket in range(len(cand_pairs[bucket_array])):
            # find all pairs from a list (if list size > 2)
            # A,B,C,D --> AB,AC,AD - BC,BD - CD
            for doc_1 in range(len(cand_pairs[bucket_array][bucket])):
                for doc_2 in range(doc_1 + 1, len(cand_pairs[bucket_array][bucket])):
                    first = cand_pairs[bucket_array][bucket][doc_1]
                    second = cand_pairs[bucket_array][bucket][doc_2]
                    # if documents are different from each other
                    if first != second:
                        # get signatures of documents from signature matrix
                        list_1 = [s[first] for s in sign_mtrx]
                        list_2 = [s[second] for s in sign_mtrx]
                        # calculate jaccard similarity
                        js = jaccard_similarity(list_1, list_2)
                        # if record / pair does not exist in similarities add it
                        if ([docs[first + 1], docs[second + 1], js] or [docs[second + 1], docs[
                        first + 1], js]) not in similarities:
                            similarities.append([docs[first + 1], docs[second + 1], js])

    # sort results
    similarities = sorted(similarities, key=lambda s: s[2], reverse=True)

    return similarities